# Sigma_density: M Dwarfs with Spectroscopic Constraints

Calculating density constraints for M Dwarf KOIs with spectroscopic constraints on properties.

1. From spectroscopy, we know Temperature, Mass, and Radius.
2. Which isochrones (stellar models, with Temperature, Mass, and Radius as parameters) fit above star?
3. Now take into account GAIA distance for the star. From this, we know the Luminosity.
4. Look back at stellar models - which isochrone fits the Temperature, Mass Radius, and Luminority (from GAIA)?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from pprint import pprint
import pandas as pd
from tqdm import tqdm
from astropy.table import Table

plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
headers = ['KOI', 'KIC', 'Teff', 'ETeff', 'eTeff', 'Fe/H', 'e_Fe/H', 'M/H', 'e_M/H', 'SpType', 'Mstar', 'e_Mstar', 'Rstar', 'e_Rstar', 'Dis', 'Fe/H-T', 'e_Fe/H-T', 'Fe/H-M', 'e_Fe/H-M', 'M/H-M', 'e_M/H-M']

For now, removed any rows with missing data and saved them into "muirhead_data_no_missing_data.txt".

In [3]:
muirhead_data = pd.read_csv("/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/Muirhead2013_isochrones/muirhead_data_no_missing_data.txt", sep=" ")

In [4]:
isochrones = pd.read_csv('/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/Muirhead2013_isochrones/isochrones_sdss_spitzer_lowmass.dat', sep='\s\s+', engine='python')

#### First, let's work with muirhead_data[0]: KIC 5868793.

1. Take Teff, Mass, and Radius.
2. Which isochrones fit these Teff, Mass, and Radius?

#### Reading Gaia-Kepler data (luminosities)

In [5]:
dat = Table.read('/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/Kepler-Gaia/kepler_dr2_4arcsec.fits', format='fits')
df = dat.to_pandas()

/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'r_result_flag' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'r_modality_flag' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'teff_err1' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'teff_err2' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
muirhead_gaia = df[df['kepid'].isin(list(muirhead_data.KIC))]
muirhead_gaia = muirhead_gaia.reset_index()
muirhead_gaia.rename(columns={"index": "KIC"})

#muirhead_comb: planet hosts with spectroscopic data + Gaia/Kepler data in one table
muirhead_comb = pd.concat([muirhead_data, muirhead_gaia], axis=1)

In [24]:
muirhead_comb_lums = muirhead_comb[muirhead_comb.lum_val.notnull()]

In [25]:
muirhead_comb_lums

,KOI,KIC,Teff,ETeff,eTeff,Fe/H,e_Fe/H,M/H,e_M/H,SpType,Mstar,e_Mstar,Rstar,e_Rstar,Dis,Fe/H-T,e_Fe/H-T,Fe/H-M,e_Fe/H-M,M/H-M,e_M/H-M,index,solution_id,designation,source_id,random_index,gaia_ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,astrometric_weight_al,astrometric_pseudo_colour,astrometric_pseudo_colour_error,mean_varpi_factor_al,astrometric_matched_observations,visibility_periods_used,astrometric_sigma5d_max,frame_rotator_object_type,matched_observations,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_nb_transits,rv_template_teff,rv_template_logg,rv_template_fe_h,phot_variable_flag,l,b,ecl_lon,ecl_lat,priam_flags,teff_val,teff_percentile_lower,teff_percentile_upper,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,flame_flags,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper,datalink_url,epoch_photometry_url,kepid,ra_kic,dec_kic,r_est,r_lo,r_hi,r_length_prior,r_result_flag,r_modality_flag,tm_designation,kepmag,teff,teff_err1,teff_err2,teff_prov,logg,logg_err1,logg_err2,logg_prov,feh,feh_err1,feh_err2,feh_prov,radius,radius_err1,radius_err2,mass,mass_err1,mass_err2,prov_sec,nconfp,nkoi,ntce,jmag,hmag,kmag,planet?,kepler_gaia_ang_dist
0,4290,5868793.0,3187.0,62.0,88.0,0.10,0.16,0.07,0.13,M4V,0.18,0.04,0.20,0.03,77.0,-0.08,0.14,0.12,0.20,-0.02,0.21,2339,1635721458409799680,b'Gaia DR2 2051797274209237376',2051797274209237376,808196649,2015.5,291.726225,0.027520,37.528968,0.035908,2.502398,0.038259,65.407204,4.990120,0.063598,11.730219,0.068544,0.022927,0.056167,-0.018181,-0.176336,0.347146,-0.276979,0.020236,-0.298436,-0.176640,-0.031674,194,0,194,0,-0.310749,182.370743,0.000000,0.000000e+00,31,False,12.063783,1.425002,0.008449,-0.035184,22,14,0.069680,0,26,False,221,10052.176541,6.280546,1600.526001,15.682715,23,3394.179959,15.449455,219.695770,16.524551,24,10018.773446,27.526410,363.969482,14.759884,1.334333,0,1.764668,0.841836,0.922832,NaN,NaN,0,NaN,NaN,NaN,b'NOT_AVAILABLE',70.276623,9.728414,304.099013,58.423520,100001,4136.250000,3980.000000,4316.479980,0.5180,0.3203,0.6026,0.2503,0.1569,0.2977,200111,0.607375,0.557714,0.656001,0.097283,0.093861,0.100705,b'http://geadata.esac.esa.int/data-server/data...,b' ',2161536,291.72620,37.528900,395.195754,389.263502,401.308280,1518.519024,1.0,1.0,b'2MASS J19265428+3731440',15.661,3967,79.0,-79.0,b'SPE70',4.674,0.032,-0.017,b'SPE60',0.06,0.15,-0.15,b'SPE70',0.587,0.026,-0.035,0.593,0.034,-0.034,b'DSEP',0,1,1,13.594,12.937,12.776,b'cand',0.063243
3,1725B,10905746.0,3253.0,62.0,57.0,0.16,0.13,0.11,0.12,M4V,0.21,0.04,0.23,0.04,34.0,0.15,0.12,0.21,0.14,0.10,0.14,4849,1635721458409799680,b'Gaia DR2 2099183583961614208',2099183583961614208,602232529,2015.5,286.594253,0.033660,37.891284,0.037412,3.421329,0.041749,81.949318,-0.822298,0.072026,6.295655,0.071103,-0.109955,-0.172098,-0.064019,0.025188,0.284440,-0.055012,-0.279648,-0.185018,-0.012759,0.075074,204,0,204,0,2.437760,250.923615,0.070480,3.665091e-01,31,False,8.492713,1.373768,0.010195,0.118008,23,16,0.074657,0,28,False,230,7730.670999,5.289180,1461.600952,15.967823,26,2179.617129,11

In [28]:
test_planet = muirhead_comb.loc[muirhead_comb['KIC'] == 11187837.0]
test_planet.lum_val

46    0.127826
Name: lum_val, dtype: float32

4   NaN
Name: lum_val, dtype: float32

In [9]:
#muirhead_comb["KIC"]

In [10]:
#muirhead_comb["lum_val"]

In [11]:
#pd.options.display.max_colwidth = 100

In [12]:
#test_planet['datalink_url']

In [13]:
def fit_isochrone(data, isochrones):
    """
       Inputs:
       data: pd.DataFrame. Spectroscopic data + Kepler/Gaia data for n stars in one table. (muirhead_comb)
       isochrones: pd.DataFrame. Isochrones table. (isochrones)
       
       Returns: list of pd.DataFrames. Each element of list is a pd.DataFrame of the isochrones that fit this star (index) BASED ONLY ON SPECTROSCOPY.
       """            
    
    iso_fits_final = list()
    
    #test each star in spectroscopy sample:
    #for i in tqdm(range(len(muirhead_comb))):
    for i in tqdm(range(1)):

        iso_fits = pd.DataFrame() 

        Teff_range = [data.Teff[i]-data.eTeff[i], data.Teff[i]+data.ETeff[i]]
        Mstar_range = [data.Mstar[i]-data.e_Mstar[i], data.Mstar[i]+data.e_Mstar[i]]
        Rstar_range = [data.Rstar[i]-data.e_Rstar[i], data.Rstar[i]+data.e_Rstar[i]]

        #test each stellar model to see if it falls within error bars:
        for j in range(len(isochrones)):
            if Teff_range[0] < 10**isochrones.logt[j] < Teff_range[1] and Mstar_range[0] < isochrones.mstar[j] < Mstar_range[1] and Rstar_range[0] < isochrones.radius[j] < Rstar_range[1]:
                iso_fits = iso_fits.append(isochrones.loc[[j]])

        iso_fits['KIC'] = muirhead_comb['KIC'][i]
        iso_fits['KOI'] = muirhead_comb['KOI'][i]

        iso_fits_final.append(iso_fits)

    return iso_fits_final


In [14]:
#isos = fit_isochrone(muirhead_comb, isochrones)

In [15]:
#isochrones['logg']

In [16]:
#for i in range(len(isos)):
#    isos[i].to_csv("isochrone_fits/spectroscopy/iso_fits_" + str([i]) + ".csv")

In [17]:
#for i in range(12, len(isos)):
#    isos[i].to_csv("isochrone_fits/spectroscopy/iso_fits_" + str(isos[i]['KIC'].iloc[0]) + ".csv")

In [18]:
def fit_isochrone_lum(data, isochrones, gaia_lum=True):
    """
       Inputs:
       data: pd.DataFrame. Spectroscopic data + Kepler/Gaia for n stars in one table. (muirhead_comb)
       isochrones: pd.DataFrame. Isochrones table. (isochrones)
       
       Returns: list of pd.DataFrames. Each element of list is a pd.DataFrame of the isochrones that fit this star (index) BASED ON SPECTROSCOPY AND GAIA LUMINOSITY.
       """    
    
    iso_fits_final = list()
    
    #for i in tqdm(range(len(muirhead_comb))):
    for i in range(1):
        
        iso_fits = pd.DataFrame() 
    
#         Teff_range = [data.Teff[i]-data.eTeff[i], data.Teff[i]+data.ETeff[i]]
#         Mstar_range = [data.Mstar[i]-data.e_Mstar[i], data.Mstar[i]+data.e_Mstar[i]]
#         Rstar_range = [data.Rstar[i]-data.e_Rstar[i], data.Rstar[i]+data.e_Rstar[i]]
#         lum_range = [data.lum_val[i]-data.lum_percentile_lower[i], data.lum_val[i]+data.lum_percentile_lower[i]]

        Teff_range = [float(data.Teff)-float(data.eTeff), float(data.Teff)+float(data.ETeff)]
        Mstar_range = [float(data.Mstar)-float(data.e_Mstar), float(data.Mstar)+float(data.e_Mstar)]
        Rstar_range = [float(data.Rstar)-float(data.e_Rstar), float(data.Rstar)+float(data.e_Rstar)]
        lum_range = [float(data.lum_val)-float(data.lum_percentile_lower), float(data.lum_val)+float(data.lum_percentile_lower)]

        print(Teff_range)
        print(Mstar_range)
        print(Rstar_range)
        print(lum_range)
        
        for j in tqdm(range(len(isochrones))):
            if gaia_lum==True:
                if Teff_range[0] < 10**isochrones.logt[j] < Teff_range[1] and Mstar_range[0] < isochrones.mstar[j] < Mstar_range[1] and Rstar_range[0] < isochrones.radius[j] < Rstar_range[1] and lum_range[0] < 10**isochrones.logl_ls[j] < lum_range[1]:
                    iso_fits = iso_fits.append(isochrones.loc[[j]])
            
            if gaia_lum==False:
                if Teff_range[0] < 10**isochrones.logt[j] < Teff_range[1] and Mstar_range[0] < isochrones.mstar[j] < Mstar_range[1] and Rstar_range[0] < isochrones.radius[j] < Rstar_range[1]:
                    iso_fits = iso_fits.append(isochrones.loc[[j]])
        
        iso_fits['KIC'] = muirhead_comb['KIC'][i]
        iso_fits['KOI'] = muirhead_comb['KOI'][i]
        
        iso_fits_final.append(iso_fits)
            
    return iso_fits_final

In [19]:
iso_lums = fit_isochrone_lum(test_planet, isochrones, gaia_lum=False)


  0%|          | 2744/738479 [00:00<00:26, 27437.21it/s]

[3137.0, 3244.0]
[0.14, 0.18]
[0.16, 0.19999999999999998]
[nan, nan]


100%|██████████| 738479/738479 [00:48<00:00, 15086.93it/s]


In [20]:
for i in range(len(iso_lums)):
   try:
       iso_lums[i].to_csv("jan29_21_iso_lums_" + '8733898' + ".csv")
   except IndexError:
       pass

In [21]:
isodf = pd.read_csv(r'/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/photoeccentric/notebooks/jan29_21_iso_lums_8733898.csv')

In [22]:
mstar = isodf["mstar"].mean()
mstar_err = isodf["mstar"].std()

rstar = isodf["radius"].mean()
rstar_err = isodf["radius"].std()

### Calculate rho

In [23]:
#Let's just do 1 star for now
ntargs = 1

In [24]:
# def find_density_dist_symmetric(ntargs, masses, masserr, radii, raderr):
#     """Gets symmetric stellar density distribution for stars.
#     Symmetric stellar density distribution = Gaussian with same sigma on each end.

#     Parameters
#     ----------
#     ntargs: int
#         Number of stars to get distribution for
#     masses: np.ndarray
#         Array of stellar masses (solar mass)
#     masserr: np.ndarray
#         Array of sigma_mass (solar mass)
#     radii: np.ndarray
#         Array of stellar radii (solar radii)
#     raderr: np.ndarray
#         Array of sigma_radius (solar radii)

#     Returns
#     -------
#     rho_dist: np.ndarray
#         Array of density distributions for each star in kg/m^3
#         Each element length 1000
#     mass_dist: np.ndarray
#         Array of symmetric Gaussian mass distributions for each star in kg
#         Each element length 1000
#     rad_dist: np.ndarray
#         Array of symmetric Gaussian radius distributions for each star in m
#         Each element length 1000
#     """

#     smass_kg = 1.9885e30  # Solar mass (kg)
#     srad_m = 696.34e6     # Solar radius (m)

#     rho_dist = np.zeros((ntargs, 1000))
#     mass_dist = np.zeros((ntargs, 1000))
#     rad_dist = np.zeros((ntargs, 1000))

#     for star in tqdm(range(ntargs)):

#         rho_temp = np.zeros(1000)
#         mass_temp = np.zeros(1000)
#         rad_temp = np.zeros(1000)

#         mass_temp = np.random.normal(masses[star]*smass_kg, masserr[star]*smass_kg, 1000)
#         rad_temp = np.random.normal(radii[star]*srad_m, raderr[star]*srad_m, 1000)

#         #Add each density point to rho_temp (for each star)
#         for point in range(len(mass_temp)):
#             rho_temp[point] = density(mass_temp[point], rad_temp[point])

#         rho_dist[star] = rho_temp
#         mass_dist[star] = mass_temp
#         rad_dist[star] = rad_temp


#     return rho_dist, mass_dist, rad_dist

In [25]:
# def density(mass, radius):
#     """Get density of sphere given mass and radius.

#     Parameters
#     ----------
#     mass: float
#         Mass of sphere (kg)
#     radius: float
#         Radius of sphere (m)

#     Returns
#     rho: float
#         Density of sphere (kg*m^-3)
#     """

#     rho = mass/((4.0/3.0)*np.pi*radius**3)
#     return rho

In [26]:
import photoeccentric as ph

In [27]:
rho, mass, radius = ph.find_density_dist_symmetric(ntargs, [mstar], [mstar_err], [rstar], [rstar_err])

100%|██████████| 1/1 [00:00<00:00, 119.37it/s]


In [28]:
np.mean(rho)

38914.793390409744

In [36]:
np.mean(mass)

3.0939090568385502e+29

In [37]:
np.mean(radius)

124848169.1645284

In [38]:
np.savetxt("jan29_21_rhos_" + '8733898' + ".csv", rho, delimiter=',')
#masspd.to_csv("jan29_21_rhos_" + str(iso_lums[i]['KIC'].iloc[0]) + ".csv")
#radiuspd.to_csv("jan29_21_rhos_" + str(iso_lums[i]['KIC'].iloc[0]) + ".csv")

In [42]:
print(mstar)
print(mstar_err)

print(rstar)
print(rstar_err)

0.15543012670105222
0.011230570640734436
0.17888163900857723
0.011918741224718215
